In [ ]:
# To be installed in EBRAINS Lab:
# %pip install --quiet --upgrade pip
# %pip install sbi==0.17.2
# %pip install --quiet -e ./tvb-inversion/mpr_sbi_tvb
# %pip install --quiet -e ./tvb-root/tvb_library
# %pip install --quiet -e ./tvb-root/tvb_storage
# %pip install --quiet -e ./tvb-root/tvb_framework

In [2]:
import os
import numpy as np
from tvb.simulator.lab import *

import sbi_tvb
from sbi_tvb.inference import TvbInference, BackendEnum
from sbi_tvb.prior import Prior
from sbi_tvb.sampler.remote_sampler import UnicoreConfig

/Users/pipeline/anaconda/anaconda3/envs/tvb-inversion/lib/python3.8/site-packages/tqdm-4.64.0-py3.8.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


   INFO  Cannot import syncrypto library.


In [2]:
sbi_tvb_path = os.path.dirname(os.path.dirname(sbi_tvb.__file__))
weights = np.loadtxt(os.path.join(sbi_tvb_path, 'data_input_files', 'SC_Schaefer7NW100p_nolog10.txt'))

model = models.MontbrioPazoRoxin(
            eta=np.r_[-4.6],
            J=np.r_[14.5],
            Delta=np.r_[0.7],
            tau=np.r_[1],)

magic_number = 124538.470647693
weights_orig = weights / magic_number
conn = connectivity.Connectivity(
            weights=weights_orig,
            region_labels=np.array(np.zeros(np.shape(weights_orig)[0]), dtype='<U128'),
            tract_lengths=np.zeros(np.shape(weights_orig)),
            areas=np.zeros(np.shape(weights_orig)[0]),
            speed=np.array(np.Inf, dtype=float),
            centres=np.zeros(np.shape(weights_orig)[0]))  # default 76 regions

integrator = integrators.HeunStochastic(
            dt=0.005,
            noise=noise.Additive(
                nsig=np.r_[0.035, 0.035 * 2],
                noise_seed=42))

_monitors = [monitors.TemporalAverage(period=0.1)]

cond_speed = np.Inf

sim = simulator.Simulator(model=model,
            connectivity=conn,
            coupling=coupling.Linear(
                a=np.r_[2.45]),
            conduction_speed=cond_speed,
            integrator=integrator,
            monitors=_monitors,
            simulation_length=30e3)

In [3]:
output_dir = os.path.join(os.getcwd(), 'results')
tvb_inference = TvbInference(sim=sim,
                             priors=[Prior('coupling.a', 1.5, 3.2)],
                             output_dir=output_dir)

In [4]:
unicore_config = UnicoreConfig(project='ich012', site='DAINT-CSCS')
tvb_inference.sample_priors(num_simulations=10, num_workers=10,
                            backend=BackendEnum.REMOTE,
                            unicore_config=unicore_config)

29-08-2022 01:13:10 - INFO - sbi_tvb.inference - Sampling priors remotely on DAINT-CSCS...
29-08-2022 01:13:10 - INFO - sbi_tvb.inference - Using dir /mnt/user/results/simulator-9uoyy4dc to store simulator with gid 8e9830cc-08bb-45e7-b7da-eb4659a47800
29-08-2022 01:13:10 - INFO - sbi_tvb.sampler.remote_sampler - Gathering files for stage in...
29-08-2022 01:13:10 - INFO - sbi_tvb.sampler.remote_sampler - Connecting to DAINT-CSCS via PyUnicore...
29-08-2022 01:13:14 - INFO - sbi_tvb.sampler.remote_sampler - Mount point on HPC for job is: /scratch/snx3000/unicore/FILESPACE/c5fd8bb3-8cb0-4491-a29d-050b860cd037/
29-08-2022 01:13:14 - INFO - sbi_tvb.sampler.remote_sampler - Starting to monitor job...
29-08-2022 01:13:14 - INFO - sbi_tvb.sampler.remote_sampler - Job has status: QUEUED
29-08-2022 01:23:15 - INFO - sbi_tvb.sampler.remote_sampler - Job has status: RUNNING
29-08-2022 01:33:15 - INFO - sbi_tvb.sampler.remote_sampler - Job has status: RUNNING
29-08-2022 01:43:15 - INFO - sbi_tvb.s

In [5]:
tvb_inference.train()

29-08-2022 01:50:09 - INFO - sbi_tvb.inference - Starting training with SNPE...
29-08-2022 01:50:09 - INFO - sbi_tvb.inference - Launching simulation...
29-08-2022 02:00:15 - INFO - sbi_tvb.inference - Finished simulation...
29-08-2022 02:00:15 - INFO - sbi_tvb.features - Computing HIGHER_MOMENTS feature...
29-08-2022 02:00:15 - INFO - sbi_tvb.features - Computing FC_CORR feature...
29-08-2022 02:00:15 - INFO - sbi_tvb.features - Computing FCD_CORR feature...
29-08-2022 02:00:16 - INFO - sbi_tvb.inference - Launching simulation...
29-08-2022 02:10:18 - INFO - sbi_tvb.inference - Finished simulation...
29-08-2022 02:10:18 - INFO - sbi_tvb.features - Computing HIGHER_MOMENTS feature...
29-08-2022 02:10:18 - INFO - sbi_tvb.features - Computing FC_CORR feature...
29-08-2022 02:10:18 - INFO - sbi_tvb.features - Computing FCD_CORR feature...
29-08-2022 02:10:18 - WARNING - root - Found 3 NaN simulations and 3 Inf simulations. They will be excluded from training.


/opt/app-root/src/.local/lib/python3.8/site-packages/sbi/neural_nets/flow.py:108: UserWarning: In one-dimensional output space, this flow is limited to Gaussians
  warn(f"In one-dimensional output space, this flow is limited to Gaussians")


Neural network successfully converged after 60 epochs.
29-08-2022 02:10:49 - INFO - sbi_tvb.inference - Finished training with SNPE...


DirectPosterior(
               method_family=snpe,
               net=<a Flow, see `.net` for details>,
               prior=BoxUniform(Uniform(low: tensor([1.5000]), high: tensor([3.2000])), 1),
               x_shape=torch.Size([1, 1404]))
               

In [6]:
data = tvb_inference.run_sim(2.2 * np.ones(1))

29-08-2022 02:10:49 - INFO - sbi_tvb.inference - Launching simulation...
29-08-2022 02:10:49 - INFO - sbi_tvb.inference - Using params: [2.2]
29-08-2022 02:20:56 - INFO - sbi_tvb.inference - Finished simulation...


In [7]:
post_samples = tvb_inference.posterior(data=data)

29-08-2022 02:20:56 - INFO - sbi_tvb.inference - Starting inference...
29-08-2022 02:20:56 - INFO - sbi_tvb.features - Computing HIGHER_MOMENTS feature...
29-08-2022 02:20:56 - INFO - sbi_tvb.features - Computing FC_CORR feature...
29-08-2022 02:20:56 - INFO - sbi_tvb.features - Computing FCD_CORR feature...


Generating samples: 100%|██████████| 1000/1000 [17:17<00:00,  1.04s/it]

29-08-2022 02:39:15 - INFO - sbi_tvb.inference - Finished inference...
29-08-2022 02:39:15 - INFO - sbi_tvb.inference - Saving posterior samples at /mnt/user/results/simulator-9uoyy4dc/posterior_samples_jn_sim.npy...
29-08-2022 02:39:15 - INFO - sbi_tvb.inference - Posterior samples mean value is 1.9954379796981812


In [8]:
print(post_samples.mean())

1.995438
